## Simulation Tests

Fixed conditions comprise of sampling time (1600 timesteps to reflect 20 minute flight in seconds), survey area (7500m x 7500m) and sensor detection zone (based on camera model). Changing variables per flight will reflect survey effort and animal availability:

1.Survey Effort

* UAV path (Lawn-mower/ Figure 8)
* FOV (BIOT/ Maldives/ Belize)
* UAV speed (lowest speed-average/ average-max speed)

2.Animal availability

* Animal movement model (Straight Line/ Stop-Start/ Random Walk)
* Animal speed (different per taxa)
* Availability bias (None/Sharks/Rays/Birds)


### Speed  Functions

* Establishing seperate speed profiles

In [1]:
from scipy.stats import truncnorm
import scipy.stats as stats
import matplotlib.pyplot as plt

def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    # More intuitive method to use truncnorm for speed_profile distribution

    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)



def speed_profile (level = 0, mean = 18, sd = 1, low = 4, upp = 33):
    # Speed profile for random sample distribution
    # Average speed: 18 m/s 
    # Level 1 = low, Level 2 = high
    
    speed_dist = get_truncated_normal(mean, sd, low, upp)
    value = speed_dist.rvs(size=1) # Select 1 random number from this distribution
    
    if level == 1:
        while value > mean:
            value = speed_dist.rvs(size=1)


    elif level == 2:
        while value < mean:
            value = speed_dist.rvs(size=1)

    else:
        value = ('You have not selected correct uav level, must be 1 or 2', 0)
    
    return value[0]
        

# Show graphics
#lower, upper = 4, 33
#mu, sigma = 18, 2
#X = stats.truncnorm(
#    (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
#N = stats.norm(loc=mu, scale=sigma)

#fig, ax = plt.subplots(2, sharex=True)
#ax[0].hist(X.rvs(10000), normed=True) # truncated normal dist
#ax[1].hist(N.rvs(10000), normed=True) # normal dist
#plt.show()

speed_profile(2)

18.976993672097045


### Speed profiles of animal

#### Elasmos & Mammals
1.  Grey reef shark (Ryan, 15)
Cruise speed - 0.64m/s | max cruise speed - 1.23 | sd - 1.23-0.35/4

2. Nurse shark (Whitney, 16)
Cruise speed is 0.33Bl/s - 0.37m/s (Mean of the mean swimming speeds in 21 - 30 degree water) ((((91+132)/2)/100)*0.33) | max - ((132/100)*0.42) (Mximum speed range with maximum body size). | sd -  (0.59 - ((91/100)*0.23))/ 4 

3. Whale shark (Jacoby,18)
Cruise speed is 0.6m/s | max - 1.06 | 1.06-0.09/4 

4. Mantaray (using for eagleray) (Fish, 18)  
Cruise speed is 1.42 m/s | max - 2.51 | 2.51-0.46/4

5. Manatee speed (Kojeszewski, 07)
Cruise speed is 0.7m/s | max - 1.14 | 1.14-0.06/4

#### Birds
1. Red-footed boobies (Weimerskirch, 05)
Median speed of 38km/h - 10.5m/s | max - 22m/s | 22-2.7/4

2. Frigatebird (Weimerskirch,03)
Average speed is 16.4km/h - 4.5 m/s | max - 5.1m/s | 5.1-2.7/4

3. Common tern average (Hedenstrom,16)
Average is 8.5 m/s | max - 10m/s | 10-7/4

#### Calculating standard deviation

Rough estimate of standard deviation derived from the range rule for standard deviation;

$$\frac {Maximum - Minimum}{4} $$

Why 4? 95% of the data will fall within 2 standard deveiations either side of the mean and so nearly all of our normal distribution will stretch over 4 standard deviations (Wan, 14).


In [2]:
from matplotlib import pyplot as plt
import statistics
import math


def animal_speed (animal):
    # Speed distribution is based on crude figures for each animal taxa
    
    speed_dict = {"reef": [0.35, 0.64, 1.23, 0.22], "nurse": [0.21, 0.37, 0.59, 0.1], "whale_shark" : [0.09, 0.6, 1.06, 0.24], "ray": [0.046, 1.42, 2.51, 0.5], 
                  "manatee": [0.06, 0.7, 1.14, 0.27], "booby": [2.7, 10.5, 22, 4.82], "frigate": [2.7, 4.5, 5.1, 0.6], "tern": [7, 8.5, 10, 0.75]}
    
    try:
        x =  speed_dict[animal.lower()]
        mean = x[1]
        upp = x[2]
        sd = x[3]
        low = x[0]
        speed_dist = get_truncated_normal(mean, sd, low, upp)
        value = speed_dist.rvs(size=1)
        return value[0]
    
    except KeyError:
        raise KeyError ('Invalid animal species, please try again')
                        
    #fig, ax = plt.subplots(1, sharex=True)
    #ax.hist(speed_dist.rvs(1000), normed=True)
    #ax.set_title(animal)
    #plt.show()
    



In [3]:
animal_speed('booby')

13.351663615654228

### UAV and Animal movement functions

1. **find_sample_points** to retrieve coordinates between two points - suitable for set UAV path and Animal straight/stop-start path. Speed level chosen according to individual and sampled randomly while angle remains consistent with direction of movement. No timsteps specified as speed of drone will dictate the number of steps between two points, i.e if the individual moves faster, it will complete the path in a shorter amount of time. Boundary points for the UAV will come into play in the path function.

2. **random_walk** runs for a specified number of timesteps for different animals (with varying speed profiles) within a specified interval of angle ranges. To prevent the individual from moving outside of the boundary, the x, y and then both x and y coordinates are inversed, changing the direction of the individual.

In [4]:
import math
import numpy as np


def round_up(n, decimals=2):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def get_speed(individual, level = 0):
    
    # Speed retrieved depending on which object we are simulating
    # individual = string of uav or animal

    if individual == 'uav':
        s = speed_profile(level)
    else:
        s = animal_speed(individual)
        
    return s


def find_sample_points(start_coord, end_coord, angle, individual, speed_level = 0, probability = 0, timestep = 1000):

    # Retrieves coordinates for x number of timesteps between 2 points
    
    # start_coord = start point of animal or drone
    # end_coord = end point of animal or drone
    # angle = angle of travel in degrees
    # individual = animal or drone?
    # speed_level = low(1) or high(2)
    # probability = probability of animal remaining stationary in that timestep(default = 0)
    
    
    horiz_dist = abs(end_coord[0] - start_coord[0]) # x coordinates
    vert_dist = abs(end_coord[1] - start_coord[1]) # y coordinates
    length = math.sqrt((horiz_dist**2) + (vert_dist**2)) # length in metres between both points
    angle_radians = math.radians(angle)

    
    total_length_covered = 0
    x_list = [start_coord[0]] # list of x values at every step
    y_list = [start_coord[1]] # list of y values at every step
    iteration = 0 # used to cumulatively add the last x, y values to recent

    while True:
        s = get_speed(individual, speed_level) # get our speed for this step
        total_length_covered += s # cumulatively add to total length covered
        #print(iteration)
        if total_length_covered > length or iteration > timestep:
            break
        
        point_probability = np.random.choice(100,1) # assign probability to coordinate
        #print(point_probability)
        if point_probability > probability:
            delta_x = x_list[iteration] + (math.cos(angle_radians)* s) # From start x coordinate, add the length with direction calculated through cosine 
            delta_y = y_list[iteration] + (math.sin(angle_radians)* s) # From start y coordinate, add the length with direction calculated through sine
          
            x = round_up(delta_x)
            y = round_up(delta_y)
            
            x_list.append(x) 
            y_list.append(y)
        
        else:
            total_length_covered = total_length_covered - s # Take last distance as we have not travelled here
            x_list.append(x_list[iteration]) 
            y_list.append(y_list[iteration])

        iteration += 1


    sample_coords = list(zip(x_list, y_list))     # zip up to create list of tuples, each being a set of coordinates where a sample has been taken
    
    
    sample_coords = sample_coords[0:timestep]
    
    
    return sample_coords 



In [82]:
test = find_sample_points([3750,0], [3750,7500], 90, 'reef', 0, 0.75)
len(test)

14.95

### UAV explicit paths

In [6]:
def detection_zone(camera):
    # FOV calculated per camera model in Strip_Sampling_Density.
    # [Height, width]
    
    camdict = {"garmin": [83,113], "sony": [114, 75], "nadir": [154, 221]}
    try:
        return camdict[camera.lower()]
    except KeyError:
        raise KeyError ('Invalid camera model used, should be :{}'.format([x for x in camdict.keys()]))

In [7]:
detection_zone('nadir')

[164, 109]

In [8]:
def boundary_coords_uav(camera, boundary_length = 5000):
    
    # Boundary_length (m)
    # x and y coordinates 
    # coord 1 = bottom righthand side (0,0)
    # coord 2 = bottom lefthand side (7500,0)
    # coord 3 = top lefthand side (7500, 7500)
    # coord 4 = top righthand side (0, 7500)
    
    area = detection_zone(camera)
    half_area = [x/2  for x in area]
    boundary = {'coord_1' : [(0 + half_area[0]), (0 + half_area[1])],
                'coord_2' : [((boundary_length) - half_area[0]), (0 + half_area[1])],
                'coord_3' : [((boundary_length) - half_area[0]), ((boundary_length) - half_area[1])],
                'coord_4' : [(0 + half_area[0]), ((boundary_length) - half_area[1])]}
    
    return boundary

#t = boundary_coords_uav('garmin')
#t

In [81]:
def lawn_shift (last_coord, camera):
    
    # Lawnmower shifts up across the virtual environment
    
    detect_area = detection_zone(camera)
    shift = detect_area[1] # Height of camera detection zone
    new_coord = [last_coord[0], last_coord[1] + shift]
    
    return new_coord
        

    

def figure_8 (camera, speed_level=1, timestep = 1000):
    
    # Figure 8 path simulation which will depend on boundary coords from camera model 
    # Returns vector of coords from simulated flight
    
    # camera is string variable - garmin or sony
    
    bc = boundary_coords_uav(camera)
    #print(bc)
    
    leg_1 = find_sample_points(bc['coord_1'], bc['coord_2'], 360, 'uav', speed_level)
    leg_2 = find_sample_points(bc['coord_2'], bc['coord_4'], 135, 'uav', speed_level)
    leg_3 = find_sample_points(bc['coord_4'], bc['coord_3'], 360, 'uav', speed_level)
    leg_4 = find_sample_points(bc['coord_3'], bc['coord_1'], 225, 'uav', speed_level)
    
    #print('Length of leg1 was:', len(leg_1))
    #print('Length of leg2 was:', len(leg_2))
    #print('Length of leg3 was:', len(leg_3))
    #print('Length of leg4 was:', len(leg_4))
    
    track = leg_1 + leg_2 + leg_3 + leg_4
    track = track[0:timestep]
    
    return track
    
    

def lawnmower (camera, speed_level = 1, timestep = 1000):
    
    # Figure 8 path simulation which will depend on boundary coords from camera model 
    # Returns vector of coords from simulated flight
    
    bc = boundary_coords_uav(camera)
    
    point1 = bc['coord_1'] # right hand side coord
    x1 = point1[0] # x value of coord that does not change throughout the changong lawn_shifts
    
    point2 = bc['coord_2'] # left hand side coord
    x2 = point2[0] # x value of coord that does not change throughout the changong lawn_shifts
    
    point3 = bc['coord_3']
    y_limit = point3[1]
    #print(y_limit)
    
    y_shifts = [0] # y value at each new shift
    right_list = []
    left_list = []
    
    while True:
        right_coord = [x1, y_shifts[-1]]
        left_coord = [x2, y_shifts[-1]]
        
        right_list.append(right_coord)
        left_list.append(left_coord)
        
        shift_coord = lawn_shift(right_coord, camera) 
        new_y = shift_coord[1]
        #print(new_y)
        
        if new_y <= y_limit:
            y_shifts.append(new_y)
        else: break
    
    right_list.pop(0) # Remove first y value which is 0
    left_list.pop(0)
    
    points = [[(0,0)],[(x1,0)]] # rough starting point for vertical shift coords to be added, two added because of iteration
    shift_points = []

    
    counter = 0
    
    for i in range(0, len(right_list)):
        #print(i)
        if i % 2 == 1:
            #print('i is odd')
            degree = 180
            leg = find_sample_points(left_list[i], right_list[i], degree, 'uav', speed_level)
           

        elif i % 2 == 0:
            #print('i is even')
            degree = 360
            leg = find_sample_points(right_list[i], left_list[i], degree, 'uav', speed_level)
          
        points.append(leg)
        #print(counter)
        counter += len(leg)
        shift = find_sample_points(points[i+1][-1], leg[0], 90, 'uav', speed_level) # estimate of distance between 
        #print('Length of shift is', len(shift))
        shift_points.append(shift)
            
        if counter >= timestep:
            break
        
  
    #shift_points.append(shift)
        
    points.pop(0) # Remove the dummy coords above
    points.pop(0)
    
    #print('Leg 5 is', len(points[5]))
    # Now to merge horizontal legs (points) with vertical shifts for each step
    # Note there are gaps in shift up and start of next leg but not by much
    
    coords = []
    
    for i in range(0, len(points)):
        coords.append(shift_points[i])
        coords.append(points[i])
    
    
    coords.pop(0) # Start from first boundary coordinate
    flat_list = []
    for sublist in coords:
        #Flatten list of lists while retaining nested lists, i.e coordinates
        for item in sublist:
            flat_list.append(item)

    flat_list = flat_list[0:timestep]       
    
    return flat_list
    
t = lawnmower('garmin', speed_level = 1, timestep = 10000)
t

[(41.5, 113),
 (57.36, 113.0),
 (74.76, 113.0),
 (92.09, 113.0),
 (109.12, 113.0),
 (126.87, 113.0),
 (143.91, 113.0),
 (161.87, 113.0),
 (179.51, 113.0),
 (196.2, 113.0),
 (213.0, 113.0),
 (230.81, 113.0),
 (248.66, 113.0),
 (266.16, 113.0),
 (283.73, 113.0),
 (301.71, 113.0),
 (319.16, 113.0),
 (335.8, 113.0),
 (352.87, 113.0),
 (370.11, 113.0),
 (387.1, 113.0),
 (402.74, 113.0),
 (420.52, 113.0),
 (437.16, 113.0),
 (453.71, 113.0),
 (470.57, 113.0),
 (487.38, 113.0),
 (504.37, 113.0),
 (521.6, 113.0),
 (539.42, 113.0),
 (556.59, 113.0),
 (573.89, 113.0),
 (591.62, 113.0),
 (608.43, 113.0),
 (626.21, 113.0),
 (643.92, 113.0),
 (660.14, 113.0),
 (677.93, 113.0),
 (694.85, 113.0),
 (712.08, 113.0),
 (729.69, 113.0),
 (746.06, 113.0),
 (762.57, 113.0),
 (780.1, 113.0),
 (797.23, 113.0),
 (814.01, 113.0),
 (830.48, 113.0),
 (847.23, 113.0),
 (864.97, 113.0),
 (881.06, 113.0),
 (898.17, 113.0),
 (915.25, 113.0),
 (932.45, 113.0),
 (949.91, 113.0),
 (967.3, 113.0),
 (984.3, 113.0),
 (1001.

### Animal paths

1. Simple movement, where S is the probability of stopping and the direction of movement is within the interval [-A, A]
 * S = 0, A = 0 (90 degrees)

2. Stop-start movement
 * S = 0.25, A = 0
 * S = 0.5, A = 0
 * S = 0.75, A = 0
 
3. Correlated random walk
 * S = 0, A = 60
 * S = 0, A = 120
 * S = 0, A = 180

In [10]:
def angle_dist(angle, size = 1):
    
    # Sampling from a uniform distribution between a chosen angle range
    
    low = -(angle)
    high = angle
    t = np.random.uniform(low = low, high = high, size = size)
    
    #plt.hist(t, bins=25, color='#75BEBB')
    #plt.xlabel('Angle (degrees)')
    #plt.ylabel('Frequency')
    #plt.xlim([low - 10, high +10])
    #plt.ylim([0, 75])
    #plt.show()
    
    return t


def random_walk (animal, start_coord, angle, timestep = 1000):
    
    # Correlated random walk for a specified animal with a start coordinate, speed profile to sample from and angle to bound uniform dist
    # x, y and then both x and y coordinates are inversed to ensure point remains within the boundary
    
    x_list = [start_coord[0]] # list of x values at every step
    y_list = [start_coord[1]] # list of y values at every step
    iteration = 0 # used to cumulatively add the last x, y values to recent
    
    
    for i in range(timestep):

            # Select angle and convert to radians
            angle_value = angle_dist(angle)
            angle_radians = math.radians(angle_value)
        
            # Select speed
            s = animal_speed(animal)
        
        
            delta_x = x_list[iteration] + (math.cos(angle_radians)* s) # From start x coordinate, add the length with direction calculated through cosine 
            delta_y = y_list[iteration] + (math.sin(angle_radians)* s) # From start y coordinate, add the length with direction calculated through sine
       
            coords = [delta_x, delta_y]
            
            test = all(i > 0 and i < 7500 for i in coords) # If out of bounds = False 
                
            if test == True: 
                #print('Correct first go:', coords)
                x = round_up(delta_x)
                y = round_up(delta_y)
            
                x_list.append(x) 
                y_list.append(y)
                
                iteration += 1
                
            else: 
                # If out of bounds reverse the x, then the y and then both x and y until you are back within the bounds
                #print('Coords before change:', coords)
                counter = 0
                while test == False:
                    counter = counter + 1
                    if counter == 1:
                        new_x = x_list[iteration] - (math.cos(angle_radians)* s)
                        coords = [new_x, delta_y]
                        #print('Coords first change:', coords)
                        test = all(i > 0 and i < 7500 for i in coords)
                    if counter == 2:
                        new_y = y_list[iteration] - (math.sin(angle_radians)* s)
                        coords = [delta_x, new_y]
                        #print('Coords second change:', coords)
                        test = all(i > 0 and i < 7500 for i in coords)
                    if counter == 3:
                        coords = [new_x, new_y]
                        #print('Coords third change:', coords)
                        test = all(i > 0 and i < 7500 for i in coords)
                        
                    
                    #print('coords passed and being appended:', coords)
                    x = round_up(coords[0])
                    y = round_up(coords[1])
            
                    x_list.append(x) 
                    y_list.append(y)
                
                    iteration += 1
                
                
     
    sample_coords = list(zip(x_list, y_list)) # zip up to create list of tuples, each being a set of coordinates where a sample has been taken
    sample_coords = sample_coords[0:timestep]
    return sample_coords 

t = random_walk('reef', [1,1], 60)
len(t)

1000

## Simulation Runs

* Run UAV and animal path combinations - categorize your runs here for later comparison
* Detection function to mark a hit and at what timestep
* Output table 

In [11]:
def perception_bias(animal):
    # Animal availability - proportion of time spent at surface
    # All obtained from http://www.penguiness.net/thebook (Laran,17)
    # Manatee: Hagihara,14
    
    biasdict = {"reef": 0.1, "nurse": 0.1, "whale_shark" : 0.6, "ray": 0.5, 
                  "manatee": 0.7, "booby": 1, "frigate": 1, "tern": 1}
    try:
        return biasdict[animal.lower()]
    except KeyError:
        raise KeyError ('Invalid animal used, should be :{}'.format([x for x in biasdict.keys()]))




def point_distance (pt1, pt2):
    # Trigonometric distance between 2 points
    
    horiz_dist = abs(pt2[0] - pt1[0]) # x coordinates
    vert_dist = abs(pt2[1] - pt1[1]) # y coordinates
    length = math.sqrt((horiz_dist**2) + (vert_dist**2)) # length in metres between both points
    
    return length



def capture (animal, animal_list, uav_list, camera, timestep = 1000, bias = 'no'):
    # Return a list of total hits and at what timestep (list of lists)
    # Capture occurs if point lies within boundary so dist between animal and uav must be < half height and half distance
    # Directly on the boarder not captured (probably wouldn't see in an actual image)
    
    # animal_list: list of coordinates
    # uav_list: list of coordinates
    # camera: camera to specify boundary - MUST BE SAME AS UAV 
    # bias: animal perception bias
    # timestep: length of simulation
    
    # Reduce list down to certain number of timesteps (remeber there will be an extra one)
    list1 = animal_list
    list2 = uav_list
    
    # Specify boundary
    area = detection_zone(camera)
    half_area = [x/2  for x in area]
    
    # Hit dict
    hits = []
    
    # List comps of distance between points and those inside boundaries
    point_dist = [point_distance(x,y) for (x,y) in zip(list1, list2)]
    point_inbounds = [(i) for (i,el) in enumerate(point_dist) if el < half_area[0] and el < half_area[1]]
    print(point_inbounds)
    if bias == 'yes' and len(point_inbounds) > 0:
        # Remove bias if specified, if not [] or [0,0]
        no_remove = int(round(len(point_inbounds)*perception_bias(animal)))
        if no_remove > 0:
            point_inbounds = point_inbounds[:-(no_remove)]
        else: pass
    else : pass
    
    return point_inbounds



list1 = find_sample_points([3750,0], [3750,7500], 90, 'nurse', 0, 0)
#print(len(list1))
list2 = figure_8('garmin')
#print(len(list2))
#camera = (10,10)
test2 = capture('nurse', list1, list2, 'garmin', bias='yes')
test2

[215, 216, 217, 218]


[215, 216, 217, 218]

In [34]:
def simulation_animal(animal, path, timestep = 1000):
    # Output vector from simulation
    # S : probability of stopping
    # A: angle
    
    # Straight
    
    
    if path == 'straight':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0, timestep)
    
    if path == 'stop25':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0.25, timestep)
    
    if path == 'stop50':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0.50, timestep)
    
    if path == 'stop75':
        vector = find_sample_points([3750,0], [3750,7500], 90, animal, 0, 0.75, timestep)
    
    if path == 'random60':
         vector = random_walk(animal, [1,1], 60, timestep)
            
    if path == 'random120':
        vector = random_walk(animal, [1,1], 120, timestep)
        
    if path == 'random180':
        vector = random_walk(animal, [1,1], 180, timestep)

    print(vector[-1])   
    return vector


simulation_animal('reef', 'stop75', 10000)

def simulation_uav(camera, path, speed, timestep = 1000):
    # Output vector for simulation
    
    if path == 'lawnmower':
        vector = lawnmower(camera, speed, timestep)
        
    if path == 'figure8':
        vector = figure_8(camera, speed, timestep)
        
    return vector
    

(3750.0, 6760.68)


In [13]:
import time

def hit_count(vect):
    
    if len(vect) > 0:
    # If not an empty hit list
        total = len(vect)
    
        hit1 = vect[:-1] # Remove last
        hit2 = vect[1:] # Remove first
    
        count = 1
    
        for i in range(0, len(hit2)):
            if(hit2[i] - hit1[i] == 1):
                count == 1 # Remains as 1 if timesteps are ensuing
            else: 
                count += 1 # Add to count score to signifiy new individual
    
        sim_score = [total, count]
        
    else:
        sim_score = [0,0]
    
    return sim_score



def simulation (animal, animal_path, camera, uav_path, speed=1, timestep = 1000, bias = 'no'):
    # Running the simulation to produce the number of hits
    
    animal_vect = simulation_animal(animal, animal_path)
    uav_vect = simulation_uav(camera, uav_path, speed)
    
    hits = capture(animal, animal_vect, uav_vect, camera, bias)
    score = hit_count(hits)
 
    #output = [uav_path, camera, speed, animal, animal_path, bias, score[0], score[1]]
    
    return score

start_time = time.time()
t = simulation('reef', 'straight', 'garmin', 'figure8', bias = 'yes')
print("--- %s seconds ---" % (time.time() - start_time))
t

[]
--- 2.34159255027771 seconds ---


[0, 0]

In [14]:
hit_count(t)

[2, 2]